In [5]:
!pip install librosa
!pip install midiutil
!pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (89 kB)
Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl (11.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
seaborn 0.13.2 requires matplotlib!=3.6.1,>=3.4, which is not installed.
mlxtend 0.16.0 requires matplotlib>=1.5.1, which is not installed.


In [7]:
import librosa
import numpy as np
import pandas as pd
import math
import soundfile as sf
import IPython.display as ipd

In [9]:
FoxTitle, sr_FoxTitle = librosa.load("music_decomposition/FoxTitle.wav", sr = 16000)
spectrogram_FoxTitle = librosa.stft(FoxTitle, n_fft=1024, hop_length=160, center=False, win_length=1024) 
M_FoxTitle = abs(spectrogram_FoxTitle)
phase_FoxTitle = spectrogram_FoxTitle/(M_FoxTitle + 2.2204e-16) 

In [11]:
U, S, V = np.linalg.svd(M_FoxTitle, full_matrices=False)

In [13]:
top_11_df = pd.DataFrame(U[:, :11])
top_11_df.to_csv("musicbases.csv", index=False)

In [15]:
n_fft = 1024
hop_length = 160
steady_start = 21
steady_end = 40

In [17]:
def compute_average_spectrum(file_path):
    y, sr = librosa.load(file_path, sr=None)
    stft_result = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    magnitude = np.abs(stft_result)
    steady_magnitude = magnitude[:, steady_start:steady_end]
    return np.mean(steady_magnitude, axis=1)

In [19]:
note_files = [
    ("A3", "music_decomposition/A3.wav"),
    ("B3", "music_decomposition/B3.wav"),
    ("C4", "music_decomposition/C4.wav"),
    ("D3", "music_decomposition/D3.wav"),
    ("D4", "music_decomposition/D4.wav"),
    ("E3", "music_decomposition/E3.wav"),
    ("E4", "music_decomposition/E4.wav"),
    ("EF4", "music_decomposition/EF4.wav"),
    ("FS3", "music_decomposition/FS3.wav"),
    ("G3", "music_decomposition/G3.wav"),
    ("G4", "music_decomposition/G4.wav")
]

In [21]:
ground_truth_spectra = {
    name: compute_average_spectrum(file_path) for name, file_path in note_files
}

In [23]:
results = []
for i, base in enumerate(U[:, :11].T):
    best_match = None
    best_inner_product = -np.inf
    for note_name, spectrum in ground_truth_spectra.items():
        inner_product = np.dot(base, spectrum)
        if inner_product > best_inner_product:
            best_match = note_name
            best_inner_product = inner_product
    results.append((i + 1, best_match, best_inner_product))

results_df = pd.DataFrame(results, columns=["Base_Index", "Note_Name", "Inner_Product"])
results_df.to_csv("matching_results.csv", index=False)

results_df

,Base_Index,Note_Name,Inner_Product
0,1,E4,-1.229586
1,2,G4,12.184568
2,3,G3,8.500744
3,4,FS3,10.305145
4,5,D3,10.522840
5,6,G4,12.573385
6,7,EF4,15.583119
7,8,EF4,18.958525
8,9,FS3,14.698790
9,10,A3,13.013588
